# Krylov Subspace Approximation XL-BOMD (KSA-XL-BOMD)
### see https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.

In [1]:
import sys
### path to PYSEQM ###
sys.path.insert(1, "/home/maxim/Projects/git2/PYSEQM_dev/")

import torch
from seqm.seqm_functions.constants import Constants
from seqm.Molecule import Molecule
from seqm.MolecularDynamics import KSA_XL_BOMD

torch.set_default_dtype(torch.float64)
if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

/home/maxim/anaconda3/envs/hipnn_1/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   279.98   1.447585e-01 -1.350261e+00 -1.205502e+00 ||    377.92   1.953992e-01 1.466852e+00 1.662251e+00 || 
     2   241.10   1.246600e-01 -1.330656e+00 -1.205996e+00 ||    338.43   1.749815e-01 1.487067e+00 1.662049e+00 || 
     3   192.51   9.953741e-02 -1.305247e+00 -1.205710e+00 ||    328.68   1.699427e-01 1.492087e+00 1.662030e+00 || 
     4   140.54   7.266691e-02 -1.277618e+00 -1.204951e+00 ||    347.27   1.795516e-01 1.482426e+00 1.661977e+00 || 
     5    91.22   4.716693e-02 -1.251883e+00 -1.204716e+00 ||    388.91   2.010823e-01 1.460577e+00 1.661659e+00 || 
     6    50.88   2.630948e-02 -1.230846e+00 -1.204537e+00 ||    445.68   2.304341e-01 1.430967e+00 1.661401e+00 || 
     7    25.21   1.303558e-02 -1.217602e+00 -1.204567e+00 ||    507.98   2.626448e-01 1.398328e+00 1.660972e+00 || 
     8    17.43   9.009704e-03 -1.213547e+00 -1.204537e+00 ||    565.87  

## Damping (Langevin)

In [3]:
%%time
torch.manual_seed(0)

species = torch.as_tensor([[8,6,1,1],[5,1,1,1]],dtype=torch.int64, device=device)[:]
coordinates = torch.tensor([
                  [
                   [ 0.00,  0.0,  0.0],
                   [ 1.22,  0.0,  0.0],
                   [ 1.82,  0.94, 0.0],
                   [ 1.82, -0.94, 0.0]
                  ],
                  [
                   [ 0.00,  0.00,  0.00],
                   [ 1.20,  0.00,  0.00],
                   [-0.60,  1.03,  0.00],
                   [-0.60, -1.03,  0.00]
                  ]
                 ], device=device)[:]

const = Constants().to(device)

elements = [0]+sorted(set(species.reshape(-1).tolist()))
seqm_parameters = {
                   'method' : 'AM1',  # AM1, MNDO, PM#
                   'scf_eps' : 1.0e-6,  # unit eV, change of electric energy, as nuclear energy doesnt' change during SCF
                   'scf_converger' : [2,0.0], # converger used for scf loop
                                         # [0, 0.1], [0, alpha] constant mixing, P = alpha*P + (1.0-alpha)*Pnew
                                         # [1], adaptive mixing
                                         # [2], adaptive mixing, then pulay
                   'sp2' : [False, 1.0e-5],  # whether to use sp2 algorithm in scf loop,
                                            #[True, eps] or [False], eps for SP2 conve criteria
                   'elements' : elements, #[0,1,6,8],
                   'learned' : [], # learned parameters name list, e.g ['U_ss']
                   #'parameter_file_dir' : '../seqm/params/', # file directory for other required parameters
                   'pair_outer_cutoff' : 1.0e10, # consistent with the unit on coordinates
                   'eig' : True
                   }


molecule = Molecule(const, seqm_parameters, coordinates, species).to(device)

# Set max_rank for a low-rank approximation. See https://pubs.acs.org/doi/full/10.1021/acs.jctc.0c00264 for details.
# Set err_threshold for low-rank approximation error tollerance. Set err_threshold=0.0 to keep max_rank constant.
# T_el - electronic temperature for thermmal HF.
# Set k for the dissipative electronic force term. See https://aip.scitation.org/doi/full/10.1063/1.3148075 for details.

output={'molid':[0, 1], 'thermo':1, 'dump':1, 'prefix':'Outputs/5_KSA_XL-BOMD_damp'}
xl_bomd_params={'k':6, 'max_rank':3, 'err_threshold':0.0, 'T_el':1500}

md = KSA_XL_BOMD(xl_bomd_params=xl_bomd_params, damp=100,
                   seqm_parameters=seqm_parameters, Temp=400.0, timestep=0.4, output=output).to(device)

md.initialize_velocity(molecule )
#remove center of mass velocity
# Info_log flag generates .txt wtih additional info for every step (orbital energies, dipole)
_ = md.run(molecule, 10, remove_com=[True, 1], Info_log=True)

Initialize velocities: zero_com
Doing initialization
Step,    Temp,    E(kinetic),  E(potential),  E(total)
     1   285.76   1.477473e-01 -1.350201e+00 -1.202454e+00 ||    380.21   1.965831e-01 1.468533e+00 1.665116e+00 || 
     2   253.02   1.308200e-01 -1.329788e+00 -1.198968e+00 ||    364.13   1.882684e-01 1.489851e+00 1.678119e+00 || 
     3   213.15   1.102080e-01 -1.302890e+00 -1.192682e+00 ||    362.56   1.874579e-01 1.496257e+00 1.683715e+00 || 
     4   172.45   8.916226e-02 -1.271479e+00 -1.182316e+00 ||    394.61   2.040309e-01 1.487860e+00 1.691891e+00 || 
     5   124.14   6.418390e-02 -1.240602e+00 -1.176418e+00 ||    458.67   2.371522e-01 1.464519e+00 1.701671e+00 || 
     6    71.01   3.671379e-02 -1.213835e+00 -1.177121e+00 ||    515.64   2.666080e-01 1.436034e+00 1.702642e+00 || 
     7    29.70   1.535480e-02 -1.197870e+00 -1.182515e+00 ||    573.25   2.963938e-01 1.405430e+00 1.701823e+00 || 
     8    13.91   7.193439e-03 -1.192542e+00 -1.185349e+00 ||    611.21  